# CS229 Project ML on Structural Type Recogntion

In [2]:
# Import Library
import numpy as np
import pandas as pd

## Data Processing

### Load data, convert numerical values

In [167]:
# Load Tax Assessor Data
df_raw = pd.read_csv('/Users/Major/Desktop/CEE299 RESEARCH\
/Raw Data/Tax Assessor data/smc_2016_zip.csv',low_memory=False)

In [3]:
## Convert numerical values from object to float 
def format_to_float(s):
    if isinstance(s, int) or isinstance(s,float):
        return float(s)
    ret = s.replace(',', '').replace('-','').lstrip().lstrip('$')
    try:
        return float(ret)
    except ValueError:
        return ret

In [4]:
# Apply function to converge all convertable columns to numerical values
lt = list(df_raw)
for i in range (len(lt)):
    df_raw[lt[i]] = df_raw[lt[i]].apply(format_to_float)
    #print('Finish working with:',lt[i])

In [5]:
# Save the raw dataframe
df_raw.to_csv('df_raw.csv', encoding='utf-8', index=False)

### Drop columns with extremly small completion and very large number of unique classification

In [11]:
# Create new df
df_feature_create = df_raw

In [12]:
# Drop columns with 0 entries
l = list(df_feature_create)
count_noetry = 0
for i in range (len(l)):
    if df_feature_create[l[i]].count() < 128137/100:
        #print('Extreme Low Completion:',l[i])
        count_noetry = count_noetry + 1
        df_feature_create=df_feature_create.drop(l[i], axis=1)
print('Number of column less than 1% complete:',count_noetry)
#df_feature_create.info()   

Number of column less than 1% complete: 39


In [13]:
# Drop columns that's classified with too many catagories
l = list(df_feature_create)
count = 0
for i in range (len(l)):
    if df_feature_create[l[i]].dtypes == 'object' :
        count_unique = len(df_feature_create[l[i]].unique())
        if count_unique > 128137 / 1000 :
            df_feature_create = df_feature_create.drop(l[i], axis=1)
            count = count + 1
print('Number of column with more than 128 unique values:',count)

Number of column with more than 128 unique values: 15


In [15]:
# Save the raw dataframe
df_feature_create.to_csv('df_feature_create.csv', encoding='utf-8', index=False)

### Seperate labeled and unlabeled examples

In [259]:
# Select Labled data
df_labeled = df_feature_create[df_feature_create['CONSTRUCTION TYPE'].notnull()]
df_labeled.to_csv('df_labeled.csv', encoding='utf-8', index=False)

In [260]:
# Unlabeled data
df_unlabeled = df_feature_create[~df_feature_create['CONSTRUCTION TYPE'].notnull()]
df_unlabeled.to_csv('df_unlabeled.csv', encoding='utf-8', index=False)

### Labled data, label and feature

In [581]:
# Dataframe for labels
df_labeled_label = pd.DataFrame()
df_labeled_label['CONSTRUCTION TYPE']= df_labeled['CONSTRUCTION TYPE']
#df_labeled_label.to_csv('df_labeled_label',encoding='utf-8', index=False)

In [582]:
# Dataframe for features
df_labeled_features = pd.DataFrame()
l = list(df_labeled)
l.remove('CONSTRUCTION TYPE')
df_labeled_features[l]=df_labeled[l]
df_labeled_features = df_labeled_features.reindex(sorted(df_labeled_features.columns), axis=1)
#df_labeled_features.to_csv('df_labeled_features',encoding='utf-8', index=False)


### Convert classification info to one hot features and normalize all features

In [70]:
## Function to expand one classified column to one hot groups
def one_hot_expand(df,column):
    l_subtitle = df[column].unique()
    for i in range(len(l_subtitle)):
        i_subtitle = str(l_subtitle[i])
        full_title = column + '_' + i_subtitle
        df[full_title] = df[column] == l_subtitle[i]

In [71]:
## Function to expand all classifies features
def expand_all(df):
    l = list(df)
    for i in range (len(l)):
        if (df[l[i]].dtypes=='object') or (l[i]=='LAND USE'):
            one_hot_expand(df,l[i])
            print('Finish Expand:',l[i])
            df = df.drop(l[i],axis=1)
    print('Finish Expand')
    return df

In [72]:
## Function to normalize numerical features
def normalize_num_column(df):
    l = list(df)
    for i in range(len(l)):
        if df[l[i]].dtypes=='float':
            mean = np.mean(df[l[i]])
            std = np.std(df[l[i]])
            df[l[i]] = (df[l[i]]-mean)/std
    print('Finish Normalize, Numerical')
    return df

In [73]:
## Function to convert to np.array
def df_to_nparray(df):
    array = df.values
    [I,J]=array.shape
    for i in range(I):
        for j in range (J):
            if array[i,j]== True:
                array[i,j]=1
            if array[i,j]== False:
                array[i,j]=0
    array = array.astype(float)
    print('Finish Convert to Array')
    return array

In [74]:
## Function to normalize all columns of array
def normalize_bool_array(array):
    l = array.shape[1]
    for i in range(l):
        mean = np.mean(array[:,i])
        std = np.std(array[:,i])
        array[:,i] = (array[:,i]-mean)/std
    print('Finish Normalize, Boolean')
    return array

In [589]:
# LABEL 
# Expand label
df_labeled_label_expand = expand_all(df_labeled_label)
# Title list
df_labeled_label_expand_title = list(df_labeled_label_expand)
# Convert to array
df_labeled_label_expand_array=df_to_nparray(df_labeled_label_expand)
# Normalized all columns in array
#df_labeled_label_expans_array_normalized= normalize_bool_array(df_labeled_label_expand_array)


Finish Expand
Finish Convert to Array


In [374]:
# Save LABEL 
df_labeled_label_expand.to_csv('df_labeled_label_expand', encoding='utf-8', index=False)
np.save('df_labeled_label_expans_array_normalized',df_labeled_label_expans_array_normalized)
with open("df_labeled_label_expand_title.txt", "w") as output:
    output.write(str(df_labeled_label_expand_title))

In [591]:
# FEATURES
# Exapand features
df_labeled_features_expand = expand_all(df_labeled_features)
# Normalize numerical features
df_labeled_features_expand_n = normalize_num_column(df_labeled_features_expand)
# Title list
df_labeled_features_expand_title = list(df_labeled_features_expand_n)
# Convert to array
df_labeled_features_expand_array = df_to_nparray(df_labeled_features_expand)
# Normailzed boolean array
df_labeled_features_expand_array_normailzed = normalize_bool_array(df_labeled_features_expand_array)
# Fill nan with 0
#df_labeled_features_expand_array_normailzed = df_labeled_features_expand_array_normailzed

Finish Expand: ABSENTEE OWNER STATUS
Finish Expand: AIR CONDITIONING
Finish Expand: BASEMENT FINISH
Finish Expand: BASEMENT TYPE CODE
Finish Expand: BLDG CODE
Finish Expand: BLDG IMPV CODE
Finish Expand: BUILDING SQUARE FEET IND
Finish Expand: CONDITION
Finish Expand: EXTERIOR WALLS
Finish Expand: FIREPLACE IND
Finish Expand: FIREPLACE TYPE
Finish Expand: FOUNDATION
Finish Expand: FRAME
Finish Expand: FUEL
Finish Expand: GARAGE
Finish Expand: HEATING
Finish Expand: HOMESTEAD EXEMPT
Finish Expand: IMPROVEMENT VALUE CALCULATED IND
Finish Expand: LAND USE
Finish Expand: LAND VALUE CALCULATED IND
Finish Expand: LOCATION INFLUENCE
Finish Expand: MAIL DIRECTION
Finish Expand: MAIL MODE
Finish Expand: MAIL STATE
Finish Expand: MAILING OPT-OUT CODE
Finish Expand: MUNICIPALITY CODE
Finish Expand: OWNER CORPORATE INDICATOR
Finish Expand: OWNER ETAL INDICATOR
Finish Expand: OWNER OWNERSHIP RIGHTS CODE
Finish Expand: OWNER RELATIONSHIP TYPE
Finish Expand: PARKING TYPE
Finish Expand: POOL
Finish Ex

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [434]:
# SAVE FEATURE
df_labeled_features_expand.to_csv('df_labeled_features_expand', encoding='utf-8', index=False)
np.save('df_labeled_features_expand_array_normailzed',df_labeled_features_expand_array_normailzed)
with open("df_labeled_features_expand_title.txt", "w") as output:
    output.write(str(df_labeled_features_expand_title))


## Seperation of data

In [50]:
# Load data
df_labeled_features_expand_array_normailzed = np.load('df_labeled_features_expand_array_normailzed.npy')
df_labeled_label_expand_array = np.load('df_labeled_label_expand_array.npy')
df_labeled_label_expand_title = ['CONSTRUCTION TYPE_WOO', 'CONSTRUCTION TYPE_CRE', 'CONSTRUCTION TYPE_1.0', 'CONSTRUCTION TYPE_FRS', 'CONSTRUCTION TYPE_SFM', 'CONSTRUCTION TYPE_FRM', 'CONSTRUCTION TYPE_MAN', 'CONSTRUCTION TYPE_BRK', 'CONSTRUCTION TYPE_MAS', 'CONSTRUCTION TYPE_CCB', 'CONSTRUCTION TYPE_STE', 'CONSTRUCTION TYPE_CUS', 'CONSTRUCTION TYPE_MET', 'CONSTRUCTION TYPE_HEA', 'CONSTRUCTION TYPE_ADB', 'CONSTRUCTION TYPE_LIG']
df_labeled_features_expand_title=['ACRES', 'APN SEQUENCE NBR', 'ASSD IMPROVEMENT VALUE', 'ASSD LAND VALUE', 'ASSD TOTAL VALUE', 'ASSESSED YEAR', 'BASEMENT SQUARE FEET', 'BEDROOMS', 'BUILDING SQUARE FEET', 'CENSUS TRACT', 'COUNTY USE1', 'COUNTY USE2', 'DEPTH FOOTAGE', 'EFFECTIVE YEAR BUILT', 'FIPS CODE', 'FIREPLACE NUMBER', 'FORMATTED APN', 'FRONT FOOTAGE', 'FULL BATHS', 'GARAGE/PARKING SQUARE FEET', 'GROSS SQUARE FEET', 'GROUND FLOOR SQUARE FEET', 'IMPROVEMENT VALUE CALCULATED', 'LAND SQUARE FOOTAGE', 'LAND USE', 'LAND VALUE CALCULATED', 'LIVING SQUARE FEET', 'MAIL HOUSE NUMBER', 'MAIL ZIP CODE', 'NUMBER OF UNITS', 'ORIGINAL APN', 'P-ID-IRIS-FRMTD', 'PARKING SPACES', 'PROPERTY INDICATOR', 'ROOF COVER', 'SITUS HOUSE NUMBER', 'SITUS UNIT NUMBER', 'SITUS ZIP CODE', 'STORIES NUMBER', 'TAX AMOUNT', 'TAX CODE AREA', 'TAX YEAR', 'TOTAL BATHS', 'TOTAL BATHS CALCULATED', 'TOTAL ROOMS', 'TOTAL VALUE CALCULATED', 'UNFORMATTED APN', 'UNITS NUMBER', 'UNIVERSAL BUILDING SQUARE FEET', 'YEAR BUILT', 'zip', 'ABSENTEE OWNER STATUS_O', 'ABSENTEE OWNER STATUS_A', 'ABSENTEE OWNER STATUS_nan', 'ABSENTEE OWNER STATUS_*', 'AIR CONDITIONING_nan', 'AIR CONDITIONING_0.0', 'AIR CONDITIONING_AWA', 'AIR CONDITIONING_1.0', 'AIR CONDITIONING_ACE', 'AIR CONDITIONING_AWI', 'AIR CONDITIONING_AEV', 'BASEMENT FINISH_nan', 'BASEMENT FINISH_FUN', 'BASEMENT FINISH_FCF', 'BASEMENT FINISH_FFI', 'BASEMENT FINISH_FPF', 'BASEMENT TYPE CODE_nan', 'BASEMENT TYPE CODE_UNF', 'BASEMENT TYPE CODE_PRL', 'BASEMENT TYPE CODE_BFL', 'BLDG CODE_nan', 'BLDG CODE_R10', 'BLDG CODE_R20', 'BLDG CODE_1.0', 'BLDG CODE_R30', 'BLDG CODE_RQ0', 'BLDG CODE_RT0', 'BLDG CODE_R80', 'BLDG CODE_R60', 'BLDG CODE_R70', 'BLDG CODE_M0H', 'BLDG CODE_999.0', 'BLDG IMPV CODE_nan', 'BLDG IMPV CODE_R10', 'BLDG IMPV CODE_R20', 'BLDG IMPV CODE_R30', 'BLDG IMPV CODE_RT0', 'BLDG IMPV CODE_R80', 'BLDG IMPV CODE_M0H', 'BLDG IMPV CODE_1.0', 'BLDG IMPV CODE_R60', 'BLDG IMPV CODE_M0L', 'BUILDING SQUARE FEET IND_L', 'BUILDING SQUARE FEET IND_B', 'BUILDING SQUARE FEET IND_nan', 'BUILDING SQUARE FEET IND_G', 'CONDITION_AVE', 'CONDITION_GOO', 'CONDITION_FAI', 'CONDITION_EXC', 'CONDITION_nan', 'CONDITION_POO', 'EXTERIOR WALLS_WOO', 'EXTERIOR WALLS_nan', 'EXTERIOR WALLS_WOS', 'EXTERIOR WALLS_WSS', 'EXTERIOR WALLS_STU', 'EXTERIOR WALLS_BRS', 'EXTERIOR WALLS_BLO', 'EXTERIOR WALLS_TUU', 'EXTERIOR WALLS_BRV', 'EXTERIOR WALLS_MET', 'EXTERIOR WALLS_ASG', 'EXTERIOR WALLS_CON', 'EXTERIOR WALLS_CNB', 'EXTERIOR WALLS_MIX', 'EXTERIOR WALLS_SWD', 'EXTERIOR WALLS_VIN', 'EXTERIOR WALLS_FRA', 'EXTERIOR WALLS_COM', 'EXTERIOR WALLS_GLA', 'EXTERIOR WALLS_1.0', 'FIREPLACE IND_Y', 'FIREPLACE IND_nan', 'FIREPLACE TYPE_1.0', 'FIREPLACE TYPE_nan', 'FIREPLACE TYPE_0U0', 'FIREPLACE TYPE_0.0', 'FIREPLACE TYPE_0W0', 'FOUNDATION_SLB', 'FOUNDATION_nan', 'FOUNDATION_RAC', 'FOUNDATION_CRE', 'FOUNDATION_MUD', 'FOUNDATION_1.0', 'FOUNDATION_MSN', 'FOUNDATION_PIR', 'FRAME_WOO', 'FRAME_1.0', 'FRAME_FPS', 'FRAME_CNS', 'FRAME_nan', 'FRAME_MSN', 'FRAME_STL', 'FRAME_CRE', 'FUEL_FGA', 'FUEL_nan', 'FUEL_FEL', 'FUEL_FWD', 'GARAGE_1.0', 'GARAGE_A00', 'GARAGE_0.0', 'GARAGE_140.0', 'GARAGE_nan', 'GARAGE_110.0', 'GARAGE_450.0', 'GARAGE_920.0', 'HEATING_1.0', 'HEATING_0.0', 'HEATING_FA0', 'HEATING_nan', 'HEATING_GR0', 'HEATING_CL0', 'HOMESTEAD EXEMPT_nan', 'HOMESTEAD EXEMPT_Y', 'IMPROVEMENT VALUE CALCULATED IND_A', 'LAND VALUE CALCULATED IND_A', 'LOCATION INFLUENCE_nan', 'LOCATION INFLUENCE_ICC', 'LOCATION INFLUENCE_ICR', 'LOCATION INFLUENCE_IWL', 'LOCATION INFLUENCE_ILK', 'LOCATION INFLUENCE_IBF', 'LOCATION INFLUENCE_IGC', 'LOCATION INFLUENCE_ILP', 'LOCATION INFLUENCE_ICU', 'LOCATION INFLUENCE_IAL', 'LOCATION INFLUENCE_I01', 'LOCATION INFLUENCE_IWF', 'LOCATION INFLUENCE_IRI', 'LOCATION INFLUENCE_ICA', 'MAIL DIRECTION_nan', 'MAIL DIRECTION_W', 'MAIL DIRECTION_N', 'MAIL DIRECTION_E', 'MAIL DIRECTION_S', 'MAIL DIRECTION_NW', 'MAIL DIRECTION_NE', 'MAIL DIRECTION_SE', 'MAIL DIRECTION_SW', 'MAIL MODE_LN', 'MAIL MODE_PL', 'MAIL MODE_RD', 'MAIL MODE_DR', 'MAIL MODE_CIR', 'MAIL MODE_nan', 'MAIL MODE_AVE', 'MAIL MODE_CT', 'MAIL MODE_ST', 'MAIL MODE_WAY', 'MAIL MODE_PKWY', 'MAIL MODE_BLVD', 'MAIL MODE_TER', 'MAIL MODE_WALK', 'MAIL MODE_EXPY', 'MAIL MODE_ISLE', 'MAIL MODE_SQ', 'MAIL MODE_IS', 'MAIL MODE_MALL', 'MAIL MODE_ROW', 'MAIL MODE_LNDG', 'MAIL MODE_RUN', 'MAIL MODE_PT', 'MAIL MODE_CMN', 'MAIL MODE_TRL', 'MAIL MODE_CV', 'MAIL MODE_LOOP', 'MAIL MODE_CTR', 'MAIL MODE_VIS', 'MAIL MODE_MNR', 'MAIL MODE_PLZ', 'MAIL MODE_HWY', 'MAIL MODE_RDG', 'MAIL MODE_PARK', 'MAIL STATE_CA', 'MAIL STATE_OH', 'MAIL STATE_WA', 'MAIL STATE_NV', 'MAIL STATE_MA', 'MAIL STATE_IN', 'MAIL STATE_TX', 'MAIL STATE_NY', 'MAIL STATE_nan', 'MAIL STATE_IL', 'MAIL STATE_FL', 'MAIL STATE_HI', 'MAIL STATE_MD', 'MAIL STATE_SD', 'MAIL STATE_NJ', 'MAIL STATE_OR', 'MAIL STATE_NM', 'MAIL STATE_MT', 'MAIL STATE_AZ', 'MAIL STATE_MN', 'MAIL STATE_VA', 'MAIL STATE_PA', 'MAIL STATE_MI', 'MAIL STATE_UT', 'MAIL STATE_CO', 'MAIL STATE_NC', 'MAIL STATE_CT', 'MAIL STATE_MO', 'MAIL STATE_ID', 'MAIL STATE_AK', 'MAIL STATE_ND', 'MAIL STATE_NE', 'MAIL STATE_KS', 'MAIL STATE_IA', 'MAIL STATE_TN', 'MAIL STATE_AL', 'MAIL STATE_GA', 'MAIL STATE_SC', 'MAIL STATE_DC', 'MAILING OPT-OUT CODE_nan', 'MAILING OPT-OUT CODE_Y', 'MUNICIPALITY CODE_06085PA0', 'MUNICIPALITY CODE_nan', 'OWNER CORPORATE INDICATOR_nan', 'OWNER CORPORATE INDICATOR_Y', 'OWNER ETAL INDICATOR_nan', 'OWNER ETAL INDICATOR_A', 'OWNER ETAL INDICATOR_U', 'OWNER OWNERSHIP RIGHTS CODE_TE', 'OWNER OWNERSHIP RIGHTS CODE_nan', 'OWNER OWNERSHIP RIGHTS CODE_RT', 'OWNER OWNERSHIP RIGHTS CODE_CP', 'OWNER OWNERSHIP RIGHTS CODE_LT', 'OWNER OWNERSHIP RIGHTS CODE_JT', 'OWNER OWNERSHIP RIGHTS CODE_TR', 'OWNER OWNERSHIP RIGHTS CODE_PT', 'OWNER OWNERSHIP RIGHTS CODE_SE', 'OWNER OWNERSHIP RIGHTS CODE_CO', 'OWNER OWNERSHIP RIGHTS CODE_CT', 'OWNER OWNERSHIP RIGHTS CODE_LE', 'OWNER OWNERSHIP RIGHTS CODE_TC', 'OWNER OWNERSHIP RIGHTS CODE_LP', 'OWNER OWNERSHIP RIGHTS CODE_UI', 'OWNER OWNERSHIP RIGHTS CODE_ES', 'OWNER OWNERSHIP RIGHTS CODE_SU', 'OWNER OWNERSHIP RIGHTS CODE_AS', 'OWNER OWNERSHIP RIGHTS CODE_SO', 'OWNER OWNERSHIP RIGHTS CODE_PS', 'OWNER OWNERSHIP RIGHTS CODE_RS', 'OWNER RELATIONSHIP TYPE_nan', 'OWNER RELATIONSHIP TYPE_HW', 'OWNER RELATIONSHIP TYPE_MA', 'OWNER RELATIONSHIP TYPE_UW', 'OWNER RELATIONSHIP TYPE_UM', 'OWNER RELATIONSHIP TYPE_SM', 'OWNER RELATIONSHIP TYPE_MM', 'OWNER RELATIONSHIP TYPE_MW', 'OWNER RELATIONSHIP TYPE_WF', 'OWNER RELATIONSHIP TYPE_SW', 'OWNER RELATIONSHIP TYPE_DV', 'OWNER RELATIONSHIP TYPE_SI', 'OWNER RELATIONSHIP TYPE_WR', 'OWNER RELATIONSHIP TYPE_DC', 'OWNER RELATIONSHIP TYPE_WD', 'OWNER RELATIONSHIP TYPE_UN', 'PARKING TYPE_nan', 'PARKING TYPE_MIX', 'PARKING TYPE_OPN', 'PARKING TYPE_COV', 'PARKING TYPE_GYY', 'PARKING TYPE_GNN', 'POOL_Y', 'POOL_nan', 'POOL CODE_300.0', 'POOL CODE_0.0', 'POOL CODE_S00', 'POOL CODE_nan', 'POOL CODE_1.0', 'POOL CODE_E00', 'POOL CODE_100.0', 'POOL CODE_00H', 'POOL CODE_A00', 'POOL CODE_00O', 'POOL CODE_M00', 'QUALITY_QAV', 'QUALITY_QGO', 'QUALITY_QEX', 'QUALITY_QPO', 'QUALITY_QFA', 'QUALITY_nan', 'QUALITY_QLU', 'SEWER_SPU', 'SEWER_nan', 'SEWER_1.0', 'SEWER_SPR', 'SEWER_SSE', 'SITUS CARRIER CODE_C059', 'SITUS CARRIER CODE_C042', 'SITUS CARRIER CODE_C016', 'SITUS CARRIER CODE_C020', 'SITUS CARRIER CODE_C058', 'SITUS CARRIER CODE_C021', 'SITUS CARRIER CODE_C061', 'SITUS CARRIER CODE_C019', 'SITUS CARRIER CODE_C030', 'SITUS CARRIER CODE_C055', 'SITUS CARRIER CODE_C026', 'SITUS CARRIER CODE_C057', 'SITUS CARRIER CODE_C033', 'SITUS CARRIER CODE_C029', 'SITUS CARRIER CODE_C037', 'SITUS CARRIER CODE_C036', 'SITUS CARRIER CODE_C022', 'SITUS CARRIER CODE_C034', 'SITUS CARRIER CODE_C035', 'SITUS CARRIER CODE_C028', 'SITUS CARRIER CODE_nan', 'SITUS CARRIER CODE_C068', 'SITUS CARRIER CODE_C024', 'SITUS CARRIER CODE_C054', 'SITUS CARRIER CODE_C011', 'SITUS CARRIER CODE_C009', 'SITUS CARRIER CODE_C049', 'SITUS CARRIER CODE_C043', 'SITUS CARRIER CODE_C005', 'SITUS CARRIER CODE_C027', 'SITUS CARRIER CODE_C003', 'SITUS CARRIER CODE_C075', 'SITUS CARRIER CODE_C053', 'SITUS CARRIER CODE_C008', 'SITUS CARRIER CODE_C070', 'SITUS CARRIER CODE_C014', 'SITUS CARRIER CODE_C004', 'SITUS CARRIER CODE_C002', 'SITUS CARRIER CODE_C017', 'SITUS CARRIER CODE_C071', 'SITUS CARRIER CODE_C007', 'SITUS CARRIER CODE_C038', 'SITUS CARRIER CODE_C010', 'SITUS CARRIER CODE_C044', 'SITUS CARRIER CODE_C041', 'SITUS CARRIER CODE_C045', 'SITUS CARRIER CODE_C074', 'SITUS CARRIER CODE_C052', 'SITUS CARRIER CODE_C046', 'SITUS CARRIER CODE_C048', 'SITUS CARRIER CODE_C072', 'SITUS CARRIER CODE_C023', 'SITUS CARRIER CODE_C032', 'SITUS CARRIER CODE_C073', 'SITUS CARRIER CODE_C031', 'SITUS CARRIER CODE_C025', 'SITUS CARRIER CODE_C078', 'SITUS CARRIER CODE_C051', 'SITUS CARRIER CODE_C040', 'SITUS CARRIER CODE_C076', 'SITUS CARRIER CODE_C001', 'SITUS CARRIER CODE_C039', 'SITUS CARRIER CODE_C050', 'SITUS CARRIER CODE_C056', 'SITUS CARRIER CODE_C006', 'SITUS CARRIER CODE_C018', 'SITUS CARRIER CODE_C013', 'SITUS CARRIER CODE_C012', 'SITUS CARRIER CODE_C015', 'SITUS CARRIER CODE_C067', 'SITUS CARRIER CODE_C066', 'SITUS CARRIER CODE_C064', 'SITUS CARRIER CODE_C047', 'SITUS CARRIER CODE_C060', 'SITUS CARRIER CODE_C065', 'SITUS CARRIER CODE_R081', 'SITUS CARRIER CODE_R777', 'SITUS CARRIER CODE_R001', 'SITUS CARRIER CODE_C086', 'SITUS CARRIER CODE_C087', 'SITUS CARRIER CODE_C085', 'SITUS CARRIER CODE_C062', 'SITUS CARRIER CODE_C091', 'SITUS CARRIER CODE_C069', 'SITUS CARRIER CODE_C063', 'SITUS CITY_PALO ALTO', 'SITUS CITY_EAST PALO ALTO', 'SITUS CITY_PORTOLA VALLEY', 'SITUS CITY_DALY CITY', 'SITUS CITY_COLMA', 'SITUS CITY_BROADMOOR VLG', 'SITUS CITY_BRISBANE', 'SITUS CITY_PACIFICA', 'SITUS CITY_SOUTH SAN FRANCISCO', 'SITUS CITY_SAN BRUNO', 'SITUS CITY_MILLBRAE', 'SITUS CITY_BURLINGAME', 'SITUS CITY_HILLSBOROUGH', 'SITUS CITY_SAN MATEO', 'SITUS CITY_MONTARA', 'SITUS CITY_MOSS BEACH', 'SITUS CITY_HALF MOON BAY', 'SITUS CITY_FOSTER CITY', 'SITUS CITY_BELMONT', 'SITUS CITY_SAN CARLOS', 'SITUS CITY_REDWOOD CITY', 'SITUS CITY_PALOMAR PARK', 'SITUS CITY_EL GRANADA', 'SITUS CITY_MENLO PARK', 'SITUS CITY_EMERALD HILLS', 'SITUS CITY_ATHERTON', 'SITUS CITY_WOODSIDE', 'SITUS DIRECTION_nan', 'SITUS DIRECTION_N', 'SITUS DIRECTION_E', 'SITUS DIRECTION_W', 'SITUS DIRECTION_S', 'SITUS MODE_LN', 'SITUS MODE_RD', 'SITUS MODE_DR', 'SITUS MODE_CT', 'SITUS MODE_AVE', 'SITUS MODE_PL', 'SITUS MODE_WAY', 'SITUS MODE_TER', 'SITUS MODE_WALK', 'SITUS MODE_ST', 'SITUS MODE_CIR', 'SITUS MODE_BLVD', 'SITUS MODE_nan', 'SITUS MODE_MNR', 'SITUS MODE_PKWY', 'SITUS MODE_VIS', 'SITUS MODE_ROW', 'SITUS MODE_HWY', 'SITUS MODE_TRL', 'SITUS MODE_ISLE', 'SITUS STATE_CA', 'STYLE_LSH', 'STYLE_USH', 'STYLE_HSH', 'STYLE_REC', 'STYLE_SPA', 'STYLE_RRA', 'STYLE_RAN', 'STYLE_TRA', 'STYLE_CNT', 'STYLE_CAP', 'STYLE_nan', 'STYLE_BUN', 'STYLE_MED', 'STYLE_VIC', 'STYLE_COL', 'STYLE_TUD', 'STYLE_GRG', 'STYLE_1.0', 'STYLE_CON', 'STYLE_AFR', 'STYLE_FRE', 'TOTAL VALUE CALCULATED IND_A', 'VIEW_nan', 'VIEW_VGO', 'VIEW_VOC', 'WATER_WPU', 'WATER_nan', 'WATER_1.0', 'WATER_0.0', 'WATER_WCO']

In [51]:
# Create new arrays
label_array = df_labeled_label_expand_array
label_title = df_labeled_label_expand_title
feature_array = np.nan_to_num(df_labeled_features_expand_array_normailzed)
feature_title = df_labeled_features_expand_title


In [52]:
# Rnadom shuffle the data set
num = label_array.shape[0]
ex = [i for i in range(num)]
np.random.shuffle(ex)
label_array_shuffled = np.empty(label_array.shape)
feature_array_shuffled = np.empty(feature_array.shape)
for i in range (num):
    label_array_shuffled[i,:] = label_array[ex[i],:]
    feature_array_shuffled[i,:] = feature_array[ex[i],:]

In [53]:
# Partition 70% data for training, 15% for validation and 15% for testing
num_dev = int(num_example*0.15)
num_test = int(num_example*0.15)
num_train = num_example - num_dev - num_test

label_dev = label_array_shuffled[0:num_dev,:]
label_test = label_array_shuffled[num_dev:num_dev+num_test,:]
label_train = label_array_shuffled[num_dev+num_test:,:]

feature_dev = feature_array_shuffled[0:num_dev,:]
feature_test = feature_array_shuffled[num_dev:num_dev+num_test,:]
feature_train = feature_array_shuffled[num_dev+num_test:,:]


## Imbalanced dataset

In [42]:
# Import 
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
## Change function to include sample weight in multiclass classifier
from sklearn.externals.joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.multiclass import _ConstantPredictor, OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer
from sklearn.linear_model import SGDClassifier

def _fit_binary_new(estimator, X, y, sample_weight, classes=None):
    unique_y = np.unique(y)
    if len(unique_y) == 1:
        if classes is not None:
            if y[0] == -1:
                c = 0
            else:
                c = y[0]
            warnings.warn("Label %s is present in all training examples." %
                          str(classes[c]))
        estimator = _ConstantPredictor().fit(X, unique_y)
    else:
        estimator = clone(estimator)

        # Only this changed
        estimator.fit(X, y, sample_weight=sample_weight)
    return estimator

class OneVsRestClassifierNew(OneVsRestClassifier):

    def fit(self, X, y, sample_weight=None):
        self.label_binarizer_ = LabelBinarizer(sparse_output=True)
        Y = self.label_binarizer_.fit_transform(y)
        Y = Y.tocsc()
        self.classes_ = self.label_binarizer_.classes_
        columns = (col.toarray().ravel() for col in Y.T)
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(delayed(_fit_binary_new)(
            self.estimator, X, column, sample_weight, classes=[
                "not %s" % self.label_binarizer_.classes_[i],
                self.label_binarizer_.classes_[i]])
            for i, column in enumerate(columns))

        return self


### Baseline

In [1030]:
# all feature set with no weight
model = OneVsRestClassifier(LogisticRegression(solver='lbfgs'))
model.fit(feature_train, label_train)
print('dev accuracy',model.score(feature_dev, label_dev))
predict = model.predict(feature_dev)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


dev accuracy 0.986171132238548


In [41]:
## Function of number of misclassified examples
def mis_cla(true,predict):
    mis = sum(abs(true-predict))
    mis_cla = sum(mis[[1,3,4,7,8,9,10]])
    return mis_cla

In [1032]:
mis = mis_cla(label_dev,predict)
print('mis clasified in types',mis)

mis clasified in types 49.0


### Over sampling

In [1040]:
# Weight
l_w = np.array([1,8,1,317,317,1,1,14,14,14,317,1,1,1,1,1])
# Duplicate examples in training set
[f1,f2]=feature_train.shape
[l1,l2]=label_train.shape
feature_train_dup = np.zeros([20647,f2])
label_train_dup = np.zeros([20647,l2])
newi = 0
for i in range(l1):
    label = label_train[i,:]
    cla = np.where(label==1)
    weight = int(l_w[cla])
    #weight = int(label_train[i,:]@l_w)
    feature_train_dup[newi:newi+weight,:] = feature_train[i,:]
    label_train_dup[newi:newi+weight,:] = label_train[i,:]
    newi=newi+weight
[feature_train_d,label_train_d]=shuffle(feature_train_dup,label_train_dup)

In [1039]:
## Function Rnadom shuffle the data set
def shuffle(feature_train_dup,label_train_dup):
    num_example = label_train_dup.shape[0]
    ex = [i for i in range(num_example)]
    np.random.shuffle(ex)
    label_train_dup_shuffled = np.empty(label_train_dup.shape)
    feature_train_dup_shuffled = np.empty(feature_train_dup.shape)
    for i in range (num_example):
        label_train_dup_shuffled[i,:] = label_train_dup[ex[i],:]
        feature_train_dup_shuffled[i,:] = feature_train_dup[ex[i],:]
    label_train_d = label_train_dup_shuffled
    feature_train_d = feature_train_dup_shuffled
    return[feature_train_d,label_train_d]

In [1033]:
# Model and prediction
model = OneVsRestClassifier(LogisticRegression(solver='lbfgs'))
model.fit(feature_train_d, label_train_d)
print('dev accuracy',model.score(feature_dev, label_dev))
predict = model.predict(feature_dev)
mis = mis_cla(label_dev,predict)
print('mis clasified in types',mis)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number 

dev accuracy 0.9625468164794008


### Weighted Samples

In [1035]:
# Sample weight calculation
w = np.array([1,8.093367347,1,317.26,317.26,1,1,13.55811966,13.55811966,13.55811966,317.26,1,1,1,1,1])    
weight_train = label_train @ w
weight_dev = label_dev @ w
             

In [1037]:
# Model with weight training and with weight predict
model = OneVsRestClassifierNew(LogisticRegression(solver='lbfgs'))
model.fit(feature_train, label_train,sample_weight=weight_train)
predict = model.predict(feature_dev)
score = model.score(feature_dev, label_dev)
print('dev score',score)
mis = mis_cla(label_dev,predict)
print('mis clasified in types',mis)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number 

dev score
mis clasified in types 106.0


### Under sampling

In [1092]:
# Only take 1960 examples form WOO and FRAM
feature_train_under = np.zeros([2298,feature_train.shape[1]])
label_train_under = np.zeros([2298,label_train.shape[1]])
count_woo = 0
count_oth = 0
count_woo_use = 0
inow = 0
for i in range(label_train.shape[0]):
    cla = label_train[i,:] @ [i for i in range(16)]
    if (cla==0) or (cla==5):
        count_woo = count_woo + 1
        if count_woo_use < 1960:
            feature_train_under[inow,:] = feature_train[i,:]
            label_train_under[inow,:] = label_train[i,:]
            inow = inow + 1
            count_woo_use = count_woo_use + 1
            
    else:
        count_oth = count_oth + 1
        feature_train_under[inow,:] = feature_train[i,:]
        label_train_under[inow,:] = label_train[i,:]
        inow = inow + 1

In [1094]:
# Suffle
[feature_train_u,label_train_u]=shuffle(feature_train_under,label_train_under)

In [1095]:
# Model and prediction
model = OneVsRestClassifier(LogisticRegression(solver='lbfgs'))
model.fit(feature_train_u, label_train_u)
print('dev accuracy',model.score(feature_dev, label_dev))
predict = model.predict(feature_dev)
mis = mis_cla(label_dev,predict)
print('mis clasified in types',mis)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


dev accuracy 0.9412273120138289
mis clasified in types 140.0


## Feature Selection 

In [37]:
# Import Chi-sqare
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

### Feature Ranking

### Chi Square

In [38]:
# num
num_label = label_array.shape[1]
num_feature = feature_array.shape[1]

# Chi square scoreto 16 label 
score_chi2 = np.empty(num_feature)
for i in range(num_label):
    t = chi2(feature_train+np.max(feature_train),label_train[:,i])
    score_chi2 = score_chi2 + t[0]
    #print('Scored Label',i)
    
# Make Ranking According to chi2 score
rank_chi2 = np.argsort(-score_chi2)

In [39]:
# Score distribution
sort_score_chi2 = -np.sort(-score_chi2)

### Mutual Information

In [1136]:
# MI square scoreto 16 label 
score_mi = np.empty(num_feature)
for i in range(num_label):
    t = mutual_info_classif(feature_train+np.max(feature_train),label_train[:,i])
    score_mi = score_mi + t
    print('Scored Label',i)
    
# Make Ranking 
rank_mi = np.argsort(-score_mi)

Scored Label 0
Scored Label 1
Scored Label 2
Scored Label 3
Scored Label 4
Scored Label 5
Scored Label 6
Scored Label 7
Scored Label 8
Scored Label 9
Scored Label 10
Scored Label 11
Scored Label 12
Scored Label 13
Scored Label 14
Scored Label 15


In [1143]:
# One off Scoreing (2h)
score_1off = np.empty(num_feature)
for i in range(num_feature):
    feature_train_new = np.delete(feature_train, i, 1)
    feature_dev_new = np.delete(feature_dev, i, 1)
    model = OneVsRestClassifierNew(LogisticRegression(solver='lbfgs'))
    model.fit(feature_train_new, label_train)
    predict = model.predict(feature_dev_new)
    score1 = model.score(feature_dev_new, label_dev)
    mis = mis_cla(label_dev,predict)
    score2 = np.exp(-(mis-49)/100);
    score_1off[i] = score1 + score2
    print('Finish score',i)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Label not 12 is present in all training examples.


Finish score 0


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Label not 12 is present in all training examples.


Finish score 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Label not 12 is present in all training examples.


...

Finish score 528


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Label not 12 is present in all training examples.


Finish score 529


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Label not 12 is present in all training examples.


Finish score 530


In [1269]:
# Make Ranking 
rank_1off = np.argsort(score_1off)

In [1323]:
# Score distribution
sort_score_1off = np.sort(score_1off)
sort_score_1off

array([2.02669381, 2.02698191, 2.02727001, 2.02755811, 2.03744223,
       2.03744223, 2.03773033, 2.03773033, 2.03773033, 2.03773033,
       2.03773033, 2.03773033, 2.03773033, 2.03773033, 2.03773033,
       2.03773033, 2.03773033, 2.03773033, 2.03801843, 2.03801843,
       2.03801843, 2.03801843, 2.04800768, 2.04800768, 2.04829578,
       2.04829578, 2.04829578, 2.04829578, 2.04829578, 2.04829578,
       2.04829578, 2.04829578, 2.04829578, 2.04829578, 2.04829578,
       2.04829578, 2.04829578, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858388,
       2.04858388, 2.04858388, 2.04858388, 2.04858388, 2.04858

### Feature Ranking Interpretation

In [1204]:
#featuregroup
feature_g = ['ACRES',\
'ABSENTEE OWNER STATUS',\
'AIR CONDITIONING',\
'APN SEQUENCE NBR',\
'ASSD IMPROVEMENT VALUE',\
'ASSD LAND VALUE',\
'ASSD TOTAL VALUE',\
'ASSESSED YEAR',\
'BASEMENT FINISH',\
'BASEMENT SQUARE FEET',\
'BASEMENT TYPE CODE',\
'BEDROOMS',\
'BLDG CODE',\
'BLDG IMPV CODE',\
'BUILDING SQUARE FEET',\
'BUILDING SQUARE FEET IND',\
'CENSUS TRACT',\
'CONDITION',\
'COUNTY USE1',\
'COUNTY USE2',\
'DEPTH FOOTAGE',\
'EFFECTIVE YEAR BUILT',\
'EXTERIOR WALLS',\
'FIPS CODE',\
'FIREPLACE IND',\
'FIREPLACE NUMBER',\
'FIREPLACE TYPE',\
'FORMATTED APN',\
'FOUNDATION',\
'FRAME',\
'FRONT FOOTAGE',\
'FUEL',\
'FULL BATHS',\
'GARAGE',\
'GARAGE/PARKING SQUARE FEET',\
'GROSS SQUARE FEET',\
'GROUND FLOOR SQUARE FEET',\
'HEATING',\
'HOMESTEAD EXEMPT',\
'IMPROVEMENT VALUE CALCULATED',\
'IMPROVEMENT VALUE CALCULATED IND',\
'LAND SQUARE FOOTAGE',\
'LAND USE',\
'LAND VALUE CALCULATED',\
'LAND VALUE CALCULATED IND',\
'LIVING SQUARE FEET',\
'LOCATION INFLUENCE',\
'MAIL DIRECTION',\
'MAIL HOUSE NUMBER',\
'MAIL MODE',\
'MAIL STATE',\
'MAIL ZIP CODE',\
'MAILING OPT-OUT CODE',\
'MUNICIPALITY CODE',\
'NUMBER OF UNITS',\
'ORIGINAL APN',\
'OWNER CORPORATE INDICATOR',\
'OWNER ETAL INDICATOR',\
'OWNER OWNERSHIP RIGHTS CODE',\
'OWNER RELATIONSHIP TYPE',\
'P-ID-IRIS-FRMTD',\
'PARKING SPACES',\
'PARKING TYPE',\
'POOL',\
'POOL CODE',\
'PROPERTY INDICATOR',\
'QUALITY',\
'ROOF COVER',\
'SEWER',\
'SITUS CARRIER CODE',\
'SITUS CITY',\
'SITUS DIRECTION',\
'SITUS HOUSE NUMBER',\
'SITUS MODE',\
'SITUS STATE',\
'SITUS UNIT NUMBER',\
'SITUS ZIP CODE',\
'STORIES NUMBER',\
'STYLE',\
'TAX AMOUNT',\
'TAX CODE AREA',\
'TAX YEAR',\
'TOTAL BATHS',\
'TOTAL BATHS CALCULATED',\
'TOTAL ROOMS',\
'TOTAL VALUE CALCULATED',\
'TOTAL VALUE CALCULATED IND',\
'UNFORMATTED APN',\
'UNITS NUMBER',\
'UNIVERSAL BUILDING SQUARE FEET',\
'VIEW',\
'WATER',\
'YEAR BUILT',\
'zip']

In [1207]:
feature_g_c = [1,
4,
7,
1,
1,
1,
1,
1,
5,
1,
4,
1,
12,
10,
1,
4,
1,
6,
1,
1,
1,
1,
20,
1,
2,
1,
5,
1,
8,
8,
1,
4,
1,
8,
1,
1,
1,
6,
2,
1,
1,
1,
23,
1,
1,
1,
14,
9,
1,
34,
39,
1,
2,
2,
1,
1,
2,
3,
21,
16,
1,
1,
6,
2,
11,
1,
7,
1,
5,
85,
27,
5,
1,
20,
1,
1,
1,
1,
21,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
3,
5,
1,
1]

In [1270]:
# Top 50 list from each method to 93 feature group
list_chi2 = list()
list_1off = list()

num = len(feature_g)

list_chi2_93count = np.zeros([num])
list_1off_93count = np.zeros([num])

def feature_group(st):
    b = st.split('_')
    return b[0]

for i in range (50):
    title_chi2 = feature_title[rank_chi2[i]]
    list_chi2.append(title_chi2)
    fg_chi2 = feature_group(title_chi2)
    index_chi2 = feature_g.index(fg_chi2)
    list_chi2_93count[index_chi2] = list_chi2_93count[index_chi2] + 1
    
    title_1off = feature_title[rank_1off[i]]
    list_1off.append(title_1off)
    fg_1off = feature_group(title_1off)
    index_1off = feature_g.index(fg_1off)
    list_1off_93count[index_1off] = list_1off_93count[index_1off] + 1

In [1254]:
trans = np.array([[1,0,0,0,0,0,0,0,0,0,],
[1,0,0,0,0,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,0,1,0,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,0,0,0,0,0,1,0,0,],
[0,0,0,0,0,0,0,1,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,0,0,0,0,0,0,0,1,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,1,],
[0,0,0,0,0,0,0,0,0,1,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,1,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,1,0,],
[0,0,0,0,1,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,1,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[1,0,0,0,0,0,0,0,0,0,],
[1,0,0,0,0,0,0,0,0,0,],
[1,0,0,0,0,0,0,0,0,0,],
[1,0,0,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,0,0,1,0,0,0,0,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,0,0,0,0,0,0,1,0,],
[0,0,0,0,0,0,0,0,1,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,1,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,0,1,0,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,0,1,0,0,0,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,1,0,0,0,0,0,],
[0,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,1,0,0,0,0,],
[0,0,0,0,0,0,0,0,1,0,],
[0,0,0,0,0,0,1,0,0,0,],
[0,0,1,0,0,0,0,0,0,0,]])

In [1271]:
list_chi2_10 = list_chi2_93count @ trans
list_1off_10 = list_1off_93count @ trans
lsit_all_10 = feature_g_c @ trans

In [1272]:
list_chi2_per = np.divide(list_chi2_10, lsit_all_10)
list_1off_per = np.divide(list_1off_10, lsit_all_10)

### Feature Ranking performance

In [1284]:
# Mannual Selection
man_151=[4,9,14,17,23,24,38,49,50,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,97,98,99,100,101,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,352,353,354,355,356,357,358,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522]
man_111=[4,9,14,17,23,24,38,49,50,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522]
man_70=[4,9,14,17,23,24,38,49,50,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]
man_62=[4,9,14,17,23,24,38,49,50,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,138,139,140,141,142,143,144,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]
man_49=[9,14,17,24,38,49,50,82,83,84,85,86,87,88,89,90,91,92,138,139,140,141,142,143,144,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]
man_38=[9,14,17,24,38,49,50,138,139,140,141,142,143,144,145,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]
man_30=[9,14,17,24,38,49,50,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]
man_16=[9,14,17,24,38,49,50,167,168,169,170,171,172,173,174,175]
man_7=[9,14,17,24,38,49,50]



In [1308]:
# Prep for scoresing
score_chi2_ac = np.zeros([9])
score_chi2_mis = np.zeros([9])
score_1off_ac = np.zeros([9])
score_1off_mis = np.zeros([9])
score_man_ac = np.zeros([9])
score_man_mis = np.zeros([9])
ls = [151,111,70,62,49,38,30,16,7]
man_list = [man_151,man_111,man_70,man_62,man_49,man_38,man_30,man_16,man_7]

def make_model_return_score(feature_list):
    # all feature set with no weight
    model = OneVsRestClassifier(LogisticRegression(solver='lbfgs'))
    model.fit(feature_train[:,feature_list], label_train)
    accuracy = model.score(feature_dev[:,feature_list], label_dev)
    predict = model.predict(feature_dev[:,feature_list])
    mis = mis_cla(label_dev,predict)
    return [accuracy, mis]
    
for i in range (9):
    n = ls[i]
    [score_chi2_ac[i],score_chi2_mis[i]] = make_model_return_score(rank_chi2[0:n])
    [score_1off_ac[i],score_1off_mis[i]] = make_model_return_score(rank_1off[0:n])
    [score_man_ac[i],score_man_mis[i]] = make_model_return_score(man_list[i])
    print('finish score',i)

    


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c

finish score 0


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c

finish score 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c

...

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))


finish score 7


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))


finish score 8


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 12 is present in all training examples.
  str(classes[c]))


## Learning Models

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [34]:
def Classify(Classifier,num_feature):
    model = Classifier
    model.fit(feature_train[:,rank_chi2[0:num_feature]], label_train)
    accuracy = model.score(feature_dev[:,rank_chi2[0:num_feature]], label_dev)
    predict = model.predict(feature_dev[:,rank_chi2[0:num_feature]])
    mis = mis_cla(label_dev,predict)
    return [accuracy, mis]

### Baseline

In [54]:
# Test for number of Features
accuracy = np.zeros([7])
mis = np.zeros([7])

for i in range(7):
    n = 10*(i+1)
    classifier = OneVsRestClassifier(LogisticRegression(solver='lbfgs'))
    [accuracy[i],mis[i]]=Classify(classifier,n)
    print('Done',n)
print('accuracy',accuracy)
print('mis',mis)

# Result
# accuracy [0.98271392 0.98501873 0.98501873 0.98473063 0.98501873 0.98559493 0.98588303]
# mis [57. 59. 59. 59. 51. 50. 46.]

Done 10
Done 20
Done 30
Done 40


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Done 50


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Done 60


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Done 70
accuracy [0.98271392 0.98501873 0.98501873 0.98473063 0.98501873 0.98559493
 0.98588303]
mis [57. 59. 59. 59. 51. 50. 46.]


### K-nearst Neighbour

In [1357]:
# Test for Hyper Parameter
accuracy = np.zeros([5])
mis = np.zeros([5])

for i in range(5):
    n = 5*(i+1)
    classifier = KNeighborsClassifier(n_neighbors=n)
    [accuracy[i],mis[i]]=Classify(classifier,70)

# result:
# accuracy = [0.98818784, 0.98789974, 0.98761164, 0.98674734, 0.98674734]
# mis = [38., 38., 41., 41., 47.]

In [1360]:
# Test for number of Features
accuracy = np.zeros([7])
mis = np.zeros([7])

for i in range(7):
    n = 10*(i+1)
    classifier = KNeighborsClassifier(n_neighbors=5)
    [accuracy[i],mis[i]]=Classify(classifier,n)
    print('Done',n)
print('accuracy',accuracy)
print('mis',mis)

# Result:
# accuracy = [0.98530683 0.98703544 0.98847594 0.98847594 0.98876404 0.98847594 0.98818784]
# mis = [49. 45. 42. 42. 38. 38. 38.]


Done 10
Done 20
Done 30
Done 40
Done 50
Done 60
Done 70
accuracy [0.98530683 0.98703544 0.98847594 0.98847594 0.98876404 0.98847594
 0.98818784]
mis [49. 45. 42. 42. 38. 38. 38.]


### DecisionTreeClassifier

In [47]:
# Test for number of Features
accuracy = np.zeros([7])
mis = np.zeros([7])
for i in range(7):
    n = 10*(i+1)
    classifier = DecisionTreeClassifier()
    [accuracy[i],mis[i]]=Classify(classifier,n)
    print('Done with feature:',n)
print('accuracy',accuracy)
print('mis',mis)

# Result
# accuracy = [0.98588303 0.98645923 0.98184961 0.98213771 0.9804091  0.97781619 0.97781619]
# mis = [46. 44. 65. 66. 70. 77. 73.] 

Done with feature: 10
Done with feature: 20
Done with feature: 30
Done with feature: 40
Done with feature: 50
Done with feature: 60
Done with feature: 70
accuracy [0.98386632 0.98559493 0.9789686  0.97723999 0.97349467 0.97723999
 0.97666379]
mis [56. 55. 79. 84. 93. 85. 76.]


### Random Forest Classifier

In [49]:
# Test for number of Features
accuracy = np.zeros([7])
mis = np.zeros([7])
for i in range(7):
    n = 10*(i+1)
    classifier = RandomForestClassifier()
    [accuracy[i],mis[i]]=Classify(classifier,n)
    print('Done with feature:',n)
print('accuracy',accuracy)
print('mis',mis)

# Result
# ac = [0.98559493 0.98703544 0.98473063 0.98645923 0.98789974 0.98847594 0.98732354]
# mis = [46. 46. 48. 46. 41. 39. 38.]

Done with feature: 10
Done with feature: 20
Done with feature: 30
Done with feature: 40
Done with feature: 50
Done with feature: 60
Done with feature: 70
accuracy [0.98473063 0.98588303 0.98415442 0.98559493 0.98588303 0.98674734
 0.98789974]
mis [54. 53. 54. 51. 49. 45. 41.]


### MLP NN

In [1365]:
# Test for number of Hidden Neurons
accuracy = np.zeros([5])
mis = np.zeros([5])
for i in range(5):
    n = 20*(i+1)
    classifier = MLPClassifier(hidden_layer_sizes=(n, ))
    [accuracy[i],mis[i]]=Classify(classifier,50)
    print('Done with feature:',n)
print('accuracy',accuracy)
print('mis',mis)

# Result
# accuracy [0.98818784 0.98876404 0.98789974 0.98732354 0.98761164]
# mis [43. 39. 39. 43. 39.]


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 20


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 40


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 60


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 80
Done with feature: 100
accuracy [0.98818784 0.98876404 0.98789974 0.98732354 0.98761164]
mis [43. 39. 39. 43. 39.]


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [1364]:
# Test for number of Features
accuracy = np.zeros([7])
mis = np.zeros([7])
for i in range(7):
    n = 10*(i+1)
    classifier = MLPClassifier()
    [accuracy[i],mis[i]]=Classify(classifier,n)
    print('Done with feature:',n)
print('accuracy',accuracy)
print('mis',mis)

# Result:
# accuracy [0.98588303 0.98818784 0.98905215 0.98789974 0.98847594 0.98761164 0.98732354]
# mis [46. 41. 40. 43. 39. 40. 41.]

Done with feature: 10
Done with feature: 20
Done with feature: 30


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 40


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 50


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Done with feature: 60
Done with feature: 70
accuracy [0.98588303 0.98818784 0.98905215 0.98789974 0.98847594 0.98761164
 0.98732354]
mis [46. 41. 40. 43. 39. 40. 41.]


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Prediction the Full Data Set

In [65]:
# The labeled dataset
labeled_label = label_array
#label_title 
labeled_feature = feature_array
#feature_title 

In [55]:
# Unlabeled dataset
df_unlabeled=pd.read_csv('df_unlabeled.csv',low_memory=False)


In [66]:
# Dataframe for features (Unlabeled)
df_unlabeled_feature = pd.DataFrame()
l = list(df_unlabeled)
l.remove('CONSTRUCTION TYPE')
df_unlabeled_feature [l] = df_unlabeled [l]
df_unlabeled_feature = df_unlabeled_feature.reindex(sorted(df_unlabeled_feature.columns), axis=1)


In [ ]:
# Expand Features (Unlabeled)
df_unlabeled_feature_expand = expand_all(df_unlabeled_feature)

# Select Feature Matches the labeled set
def match_features(full_df):
    new_df = pd.DataFrame()
    for i in range (len(feature_title)):
        feature = feature_title[i]
        
        try:
            new_df[feature]=full_df[feature]
        except KeyError:
            new_df[feature]=0
    return new_df
    
df_unlabeled_feature_matched = match_features(df_unlabeled_feature_expand)

# Normalize numerical features
df_unlabeled_features_expand_matched_n = normalize_num_column(df_unlabeled_feature_matched)


In [148]:
# Convert to array
t = df_unlabeled_features_expand_matched_n
array = t.values
[I,J]=array.shape
for i in range(I):
    for j in range(J):
        if array[i,j]==True:
            array[i,j]=1
        if array[i,j]==False:
            array[i,j]=0
unlabeled_features_expand_array = array.astype(float)

# Fill nan with 0 
unlabeled_features_expand_array_filled = np.nan_to_num(unlabeled_features_expand_array)

# Normailzed boolean array
unlabeled_features_expand_array_filled_normalized = normalize_bool_array(unlabeled_features_expand_array_filled)
unlabeled_feature = np.nan_to_num(unlabeled_features_expand_array_filled_normalized)

array([[-0.04314867,         nan, -0.05306803, ...,  0.        ,
         0.        ,  0.        ],
       [-0.04404542,         nan, -0.02860359, ...,  0.        ,
         0.        ,  0.        ],
       [-0.0396519 ,         nan,  0.34576464, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.02425528,         nan, -0.09870756, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02425528,         nan, -0.1618187 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.02425528,         nan, -0.08097395, ...,  0.        ,
         0.        ,  0.        ]])

In [156]:
# Train with the full labeled data
    # KNearest-Neighbour, k=5, chi2 rank top 50 features
model = KNeighborsClassifier(n_neighbors=5)
model.fit(labeled_feature[:,rank_chi2[0:50]], labeled_label)
print('Finish Training')
predict = model.predict(unlabeled_feature[:,rank_chi2[0:50]])
print('Finish Prediction')

# Result
sum(predict)
# [2.89000e+02, 1.20000e+01, 0.00000e+00, 0.00000e+00, 0.00000e+00,
#        1.02901e+05, 0.00000e+00, 1.42600e+03, 1.57000e+02, 0.00000e+00,
#        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
#        0.00000e+00]

Finish Training
Finish Prediction


array([2.89000e+02, 1.20000e+01, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       1.02901e+05, 0.00000e+00, 1.42600e+03, 1.57000e+02, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00])

In [165]:
# Train with the full labeled data
    # NN, 20 Neurons, chi2 rank top 50 features
model = MLPClassifier(hidden_layer_sizes=(n, ))
model.fit(labeled_feature[:,rank_chi2[0:50]], labeled_label)
print('Finish Training')
predict = model.predict(unlabeled_feature[:,rank_chi2[0:50]])
print('Finish Prediction')
# Result
sum(predict)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Finish Training
Finish Prediction


array([  583,  1326,     0,     0,     0, 92941,     0,  2217,    44,
          27,     0,     0,     0,     0,     1,     0])

In [158]:
# Train with the full labeled data
    # NN, 20 Neurons, chi2 rank top 50 features
model = MLPClassifier(hidden_layer_sizes=(n, ), activation='logistic')
model.fit(labeled_feature[:,rank_chi2[0:50]], labeled_label)
print('Finish Training')
predict = model.predict(unlabeled_feature[:,rank_chi2[0:50]])
print('Finish Prediction')

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Finish Training
Finish Prediction


## Reality Check

In [170]:
df_raw['LAND USE'].unique()

array([163, 244, 211, 675, 112, 300, 364, 879, 352, 324, 400, 652, 266,
       320, 200, 133, 115, 600, 308, 316, 283, 655, 100, 132, 700, 151,
       660, 281, 252, 278, 205, 742, 877, 261, 223, 301, 165, 270, 279,
       795, 102, 212, 725, 240, 326, 342, 349, 157, 610, 229, 606, 127,
       358, 401, 880, 230, 217, 136, 755, 480, 286, 757, 304, 334, 712,
       164, 249, 430, 502, 111, 680, 500, 560, 511, 766, 886, 540, 137])

In [176]:
sum(df_raw['LAND USE']==675)
# Single Family Residential - 100834
# Office - 929
# COMMERCIAL BUILDING - 211
# RELIGIOUS - 232
# 


232